## Relational Datastore

In [10]:
pip install pymysql

In [2]:
import json
import pandas as pd
import pymysql
import datetime
import time

User data overview

In [3]:
lines=0
num_tweets=0
num_retweets=0
print_tweet = 0
print_retweet = 0
tweets = {}

#insert path and replace name of the file below as needed
with open("corona-out-3", "r") as f1:
    for line in f1:
        try:
            data = json.loads(line)
            lines = lines + 1
            if ( data['text'].startswith('RT') ):
                num_retweets += 1
                
            else:
                num_tweets += 1
                
            if (data['id_str'] not in tweets):
                tweets[data['id_str']] = data
            
        except:
            # if there is an error loading the json of the tweet, skip
            continue
            
print('num of lines=',lines, 'num of tweets=',num_tweets, 'num of retweets=',num_retweets)
print('num of unique tweets/retweets=',len(tweets.keys()))

num of lines= 101916 num of tweets= 40804 num of retweets= 61112
num of unique tweets/retweets= 101894


In [10]:
# print user information of the last data item
user = data['user']
print('There are', len(user), 'features in user data.')
user

There are 38 features in user data.


{'id': 933734367679123456,
 'id_str': '933734367679123456',
 'name': 'Jacob',
 'screen_name': 'lundgreenjacob',
 'location': None,
 'url': None,
 'description': None,
 'translator_type': 'none',
 'protected': False,
 'verified': False,
 'followers_count': 1,
 'friends_count': 44,
 'listed_count': 0,
 'favourites_count': 68,
 'statuses_count': 12,
 'created_at': 'Thu Nov 23 16:29:56 +0000 2017',
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': False,
 'lang': None,
 'contributors_enabled': False,
 'is_translator': False,
 'profile_background_color': 'F5F8FA',
 'profile_background_image_url': '',
 'profile_background_image_url_https': '',
 'profile_background_tile': False,
 'profile_link_color': '1DA1F2',
 'profile_sidebar_border_color': 'C0DEED',
 'profile_sidebar_fill_color': 'DDEEF6',
 'profile_text_color': '333333',
 'profile_use_background_image': True,
 'profile_image_url': 'http://pbs.twimg.com/profile_images/933736718141968384/uxpf4THz_normal.jpg',
 'profile_image_url_htt

Create new database *db1*

In [4]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', charset="utf8")
cur = conn.cursor()

In [5]:
cur.execute("CREATE DATABASE IF NOT EXISTS db1")
conn.commit()

In [6]:
cur.close()
conn.close()

Create *users* table

In [20]:
conn = pymysql.connect(host='127.0.0.1', port=3306, user='root', charset="utf8mb4", database='db1')
cur = conn.cursor()

In [6]:
cur.execute("SHOW VARIABLES LIKE 'character_set_database'")
result = cur.fetchone()
print(f"Database charset: {result[1]}")

cur.execute("ALTER DATABASE db1 CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci")
conn.commit()

Database charset: utf8mb4


In [7]:
cur.execute("""
CREATE TABLE IF NOT EXISTS users (
    id_str VARCHAR(255) PRIMARY KEY,
    screen_name VARCHAR(255),
    name VARCHAR(255),
    created_at DATETIME,
    protected BOOLEAN,
    favourites_count INT,
    statuses_count INT
) CHARACTER SET utf8mb4 COLLATE utf8mb4_general_ci
""")

conn.commit()

Insert data into the *users* table

In [10]:
# Do not run this code.
def insert_user_into_table(user):
    created_at = datetime.datetime.strptime(user['created_at'], "%a %b %d %H:%M:%S %z %Y")
    formatted_created_at = created_at.strftime("%Y-%m-%d %H:%M:%S")
    
    sql = """
    INSERT INTO users (id_str, screen_name, name, created_at, protected, favourites_count, statuses_count) 
    VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
        screen_name = VALUES(screen_name),
        name = VALUES(name),
        created_at = VALUES(created_at),
        protected = VALUES(protected),
        favourites_count = VALUES(favourites_count),
        statuses_count = VALUES(statuses_count)
    """
    cur.execute(sql, (user['id_str'], user['screen_name'], user['name'], formatted_created_at, 
                      user['protected'], user['favourites_count'], user['statuses_count']))
    conn.commit()


seen_tweets = set()
seen_users = set()

with open("corona-out-3", "r") as f1:
    for line in f1:
        if line.strip() == "":
            continue
        
        try:
            data = json.loads(line)

            if data['id'] in seen_tweets:
                continue
            else:
                seen_tweets.add(data['id'])
            
            user = data['user']

            if user['id_str'] not in seen_users:
                seen_users.add(user['id_str'])

                insert_user_into_table(user)
        
        except Exception as e:
            print("Error:", e)
            continue

In [19]:
sql = "SELECT * FROM users LIMIT 10"
cur.execute(sql)
cur.fetchall()

(('1000006582896295938', 'yourgirlsarag', 'sara', 11349, 27070),
 ('1000027886915637250', 'cheche04125843', 'cheche', 1031, 5628),
 ('1000034375973646337', 'clarice8967', 'clarih 🐻', 40141, 11868),
 ('100004211', 'karundhiravidan', 'azakiya tamilmagan', 70330, 35871),
 ('1000045322859634688', 'barcelona9581', 'barcelona958', 29449, 14216),
 ('1000046756120363013', 'isisevang', 'áisis', 9384, 8155),
 ('1000047984103575552',
  'esesesrarara34',
  'EsRaAa (EN YENI HESAP)😂',
  1459,
  1170),
 ('1000055605263065088', 'zolesa_ke', 'Tsika Zolesa', 37216, 23844),
 ('1000061119921352705', 'lejayk54', '☮️Axel☮️', 1438, 630),
 ('1000062434126725120', 'mahin37418807', 'Mahin Mohades', 5232, 4104))

Check *protected* property

In [17]:
# Do not run this code.
cur.execute("SELECT * FROM users WHERE protected = 1")
cur.fetchall()

()


For all of the users, *protected* values are 0.

Dropping irrelevant fields

In [14]:
# Do not run this code.
cur.execute("ALTER TABLE users DROP COLUMN protected;")
conn.commit()

In [17]:
cur.execute("ALTER TABLE users DROP COLUMN created_at;")
conn.commit()

Check whether *screen_name* or *name* is unique

In [11]:
cur.execute("SELECT screen_name, COUNT(*) FROM users GROUP BY screen_name HAVING COUNT(*) > 1;")

0

In [12]:
cur.execute("SELECT name, COUNT(*) FROM users GROUP BY name HAVING COUNT(*) > 1;")

2470

Rename *id_str*

In [20]:
cur.execute("ALTER TABLE users DROP PRIMARY KEY;")
cur.execute("ALTER TABLE users CHANGE id_str userid_str VARCHAR(255) PRIMARY KEY;")
conn.commit()

Queries

In [21]:
cur.execute("""
SELECT userid_str, name, screen_name, favourites_count FROM users
ORDER BY favourites_count desc
""")

result1 = cur.fetchall()

In [22]:
column_names1 = [desc[0] for desc in cur.description]
df1 = pd.DataFrame.from_records(result1, columns=column_names1)
df1.head(10)

,userid_str,name,screen_name,favourites_count
0,752559084881211392,ronnie cassol,ronniehowlett3,1105807
1,1672748778,Die Mim,kulturbolschewi,1081511
2,2184465813,dieSchatzen Oma,Tirolerin45,1002202
3,3232991391,Chamaelita™@🏡,Chamaelita,974476
4,230310402,domenico fors,Domenico1oo777,873692
5,703311650,J.P.,Wuchtbrumme80,857564
6,94095624,Suryakant,suryakantvsnl,843480
7,153177381,MiJD,micjojd,837447
8,959105130955247616,sawati dasi💧,PitaRampal,835740
9,864487498935787520,Rhonda Harbison,rhonda_harbison,828722


In [23]:
cur.execute("""
SELECT userid_str, name, screen_name, statuses_count FROM users
ORDER BY statuses_count desc
""")

result2 = cur.fetchall()

In [24]:
column_names2 = [desc[0] for desc in cur.description]
df2 = pd.DataFrame.from_records(result2, columns=column_names2)
df2.head(10)

,userid_str,name,screen_name,statuses_count
0,1179710990,OccuWorld 🏴,OccuWorld,3501276
1,289118612,Tomthunkit™,TomthunkitsMind,2401909
2,317671787,informazione interno,infoitinterno,2296984
3,113410319,Vitor Santos,jornalistavitor,2211755
4,292619181,scarlet monahan,scarletmonahan,1904571
5,61608747,Real Marsha Wright® | #StaySafeBeWiseLoveEachO...,marshawright,1803883
6,47274731,Okezone,okezonenews,1667748
7,69183155,detikcom,detikcom,1638984
8,18040227,Ratno Wistu Jr,RatnoWistu,1638089
9,2359140223,जॉर्ज कुरियन,GeorgekurianINC,1537283


Add an Index on *screen_name*

In [8]:
cur.execute('''
ALTER TABLE users
ADD INDEX screen_name_idx (screen_name)
''')
conn.commit()

In [21]:
cur.close()
conn.close()